## Set **seed**

In [ ]:
from sefixlines.utils import set_all_seeds

set_all_seeds()

## Data

In [ ]:
import albumentations as A
from sefixlines.datasets import ImageSemanticSegmentationDataset

### **Initial**

In [ ]:
image_paths = []
mask_paths = []

### **Split**

In [ ]:
from sklearn.model_selection import train_test_split

train_image_paths, valid_image_paths, train_mask_paths, valid_mask_paths = train_test_split(image_paths, mask_paths, test_size=0.2, random_state=42)

### **Augmentations**

In [ ]:
ImageSemanticSegmentationDataset.augmentation = A.Compose([
    # Добавьте своих аугментаций (по желанию)
])

### Create **Datasets**

In [ ]:
dataset = ImageSemanticSegmentationDataset(image_paths, mask_paths)

train_set = ImageSemanticSegmentationDataset(train_image_paths, train_mask_paths, True)
valid_set = ImageSemanticSegmentationDataset(valid_image_paths, valid_mask_paths)

### ***Visualization***

In [ ]:
dataset.show()

## **Models**

In [ ]:
from torch import nn, optim
from sefixlines.models import SemanticSegmenter

### *Score*

In [ ]:
scores = dict()

### **Model**: `Unet`

In [ ]:
model_image_size = (224, 224)
ImageSemanticSegmentationDataset.change_image_size(model_image_size)

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    in_channels=3, 
    classes=2
)

In [ ]:
model_wrapped = SemanticSegmenter(model, "Unet")
model_wrapped.fit(train_set, valid_set, num_epochs=3)

In [ ]:
scores[model_wrapped.best_score] = model_wrapped
model_wrapped.visualize_segmentation(valid_set)

## Result

In [ ]:
best_model_wrapped = scores[max(scores)]
best_model_wrapped.name

## Submission

In [ ]:
test_dir = ""

In [ ]:
import os

test_image_names = os.listdir(test_dir)
test_image_paths = [f"{test_dir}/{image_name}" for image_name in test_image_names]
test_set = ImageSemanticSegmentationDataset(test_image_paths)

In [ ]:
prediction_mask = best_model_wrapped.predict(test_set)